In [148]:
from bs4 import BeautifulSoup
import pandas as pd
import requests
import time

In [149]:
url = 'https://myanimelist.net/topanime.php'
page = requests.get(url)

soup = BeautifulSoup(page.text,'html.parser')

In [150]:
top100 = soup.find('table',class_ ='top-ranking-table')
header = top100.find('tr')
header_titles = header.find_all('td')
table_header_titles = [title.text for title in header_titles]
print(table_header_titles)
table_header_titles = table_header_titles[0:3]
print(table_header_titles)
table_header_titles.insert(2,"Members")
table_header_titles.insert(2,"Date_end")
table_header_titles.insert(2,"Date_start")
table_header_titles.insert(2,"Type_count")
table_header_titles.insert(2,"Type")
print(table_header_titles)

['Rank', 'Title', 'Score', 'Your Score', 'Status']
['Rank', 'Title', 'Score']
['Rank', 'Title', 'Type', 'Type_count', 'Date_start', 'Date_end', 'Members', 'Score']


In [151]:
top100_animes=[]
for i in range(0,3000,50):
    url = f'https://myanimelist.net/topanime.php?limit={i}'
    time.sleep(10)
    page = requests.get(url)
    soup = BeautifulSoup(page.text,'html.parser')
    top100 = soup.find('table',class_ ='top-ranking-table')
    data_set = top100.find_all('tr',class_='ranking-list')
    for data in data_set:
        anime_data_row = []
        anime_data_row.append(data.find('td',class_='rank ac').get_text().strip())
        data_title =data.find('td',class_='title al va-t word-break').get_text().strip()
        
        splitted_data = data_title.split('\n')
        for counter,splitted_data_ in enumerate(splitted_data):
            if counter == 0:
                 anime_data_row.append(splitted_data_.strip())
            elif counter == 1:
                splitted_type = splitted_data_.split('(')
                anime_data_row.append(splitted_type[0].strip())
                anime_data_row.append(splitted_type[1].replace(')','').replace('eps','').strip())
            elif counter == 2:
                splitted_date = splitted_data_.split('-')
                anime_data_row.append(splitted_date[0].strip())
                if splitted_date[1] != '':
                    anime_data_row.append(splitted_date[1].strip())
            elif counter == 3:
                anime_data_row.append(splitted_data_.replace('members','').strip())
            else:
                break
                
        anime_data_row.append(data.find('td',class_='score ac fs14').get_text().strip())
        top100_animes.append(anime_data_row)
        

In [152]:
df = pd.DataFrame(top100_animes,columns=table_header_titles)
df

,Rank,Title,Type,Type_count,Date_start,Date_end,Members,Score
0,1,Sousou no Frieren,TV,28,Sep 2023,Mar 2024,"557,553",9.14
1,2,Fullmetal Alchemist: Brotherhood,TV,64,Apr 2009,Jul 2010,"3,310,183",9.09
2,3,Steins;Gate,TV,24,Apr 2011,Sep 2011,"2,538,564",9.07
3,4,Gintama°,TV,51,Apr 2015,Mar 2016,"624,001",9.06
4,5,Shingeki no Kyojin Season 3 Part 2,TV,10,Apr 2019,Jul 2019,"2,244,066",9.05
...,...,...,...,...,...,...,...,...
2995,2996,Hengen Taima Yakou Karura Mau! Sendai Kokeshi ...,OVA,6,Sep 1990,Mar 1991,"1,510",7.24
2996,2997,High School DxD Hero,TV,12,Apr 2018,Jul 2018,"503,734",7.24
2997,2998,Beelzebub-jou no Okinimesu mama.,TV,12,Oct 2018,Dec 2018,"96,428",7.23
2998,2999,Boku no Hero Academia: Sukue! Kyuujo Kunren!,OVA,1,Apr 2017,Apr 2017,"180,091",7.23


In [153]:
df.to_csv('~/PycharmProjects/pythonProject/top100_animes.csv')